In [ ]:
!pip install scikit-learn==1.0 gunicorn

     |████████████████████████████████| 23.1 MB 1.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# download model and preprocessor files
PREFIX='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework'
!wget $PREFIX/model1.bin
!wget $PREFIX/dv.bin

--2021-10-08 09:16:31--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     746  --.-KB/s    in 0s      

2021-10-08 09:16:31 (12.2 MB/s) - ‘model1.bin’ saved [746/746]

--2021-10-08 09:16:31--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP

# Question 1
* Install `Pipenv`
* What's the version of pipenv you installed?
* Use `--version` to find out

### answer: 2021.5.29

# Question 2
* Use `Pipenv` to install `scikit-Learn` version 1.0
* What's the first hash for `scikit-learn` you get in `Pipfile.lock`?

### answer: sha256:121f78d6564000dc5e968394f45aac87981fcaaf2be40cfcd8f07b2baa1e1829

# Question 3
Let's use these models!

* Write a script for loading these models
* Score this customer:
`{"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}`
* What's the probability that this customer is churning?

In [2]:
import pickle

In [ ]:
model_file = 'model1.bin'
dv_file = 'dv.bin'

customer = {"contract": "two_year", 
            "tenure": 12, 
            "monthlycharges": 19.7}

with open(model_file, 'rb') as f:
    model = pickle.load(f)

with open(dv_file, 'rb') as f:
    dv = pickle.load(f)

X = dv.transform(customer)
prediction = model.predict_proba(X)[0,1]
print(f'Probability of churning: {prediction:.3f}')

Probability of churning: 0.115


### answer: 0.115

# Question 4
Now let's serve this model as a web service

* Install Flask and Gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this customer using requests:

```
url = "YOUR_URL"
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
requests.post(url, json=customer).json()
```
What's the probability that this customer is churning?

In [ ]:
import requests
url = 'http://localhost:9696/predict'
customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}


Here use a Google Colab terminal with commands:
```
cd '/content/gdrive/My Drive/Github/MLZoomcamp2021/Homework5/'
gunicorn --bind 0.0.0.0:9696 predict:app
```



In [ ]:
requests.post(url, json=customer).json()

{'churn': True, 'churn_probability': 0.9988892771007961}

### answer: 0.998

# Docker
Install Docker. We will use it for the next two questions.

For these questions, I prepared a base image: `agrigorev/zoomcamp-model:3.8.12-slim`. You'll need to use it (see Question 5 for an example).

This image is based on `python:3.8.12-slim` and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

```
FROM python:3.8.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```
I already built it and then pushed it to [`agrigorev/zoomcamp-model:3.8.12-slim`](https://hub.docker.com/r/agrigorev/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.



# Question 5
Now create your own Dockerfile based on the image I prepared.

It should start like that:

```
FROM agrigorev/zoomcamp-model:3.8.12-slim
# add your stuff here
```
Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with gunicorn

When you build your image, what's the digest for `agrigorev/zoomcamp-model:3.8.12-slim`?

Look at the first step of your build log. It should look something like that:

```
Step 1/3 : FROM python:3.8.12-slim
 ---> 2e56f6b0af69
```

### answer f0f43f7bc6e0

# Question 6
Let's run your docker container!

After running it, score the same customer:

```
url = "YOUR_URL"
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}
requests.post(url, json=customer).json()
```

What's the probability that this customer is churning?

### answer 0.728

<blockquote class="twitter-tweet"><p lang="en" dir="ltr"><a href="https://twitter.com/hashtag/DataTalksClub?src=hash&amp;ref_src=twsrc%5Etfw">#DataTalksClub</a> <a href="https://twitter.com/hashtag/MLZoomcamp?src=hash&amp;ref_src=twsrc%5Etfw">#MLZoomcamp</a> homework #5 - model deployment - took more time as expected...</p>&mdash; sha of smile (@trueRock_n_roll) <a href="https://twitter.com/trueRock_n_roll/status/1446779444597858309?ref_src=twsrc%5Etfw">October 9, 2021</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>